In [ ]:
# Run this to setup environment
%pip install -r requirements.txt

In [3]:
import pandas as pd
from processing_utils import *
from rbf import RBF
from rbf_optimization import objective
import optuna
from sklearn.metrics import r2_score, mean_squared_error
import random
%reload_ext autoreload
%autoreload 2

### RBF Network Regression

In [4]:
raw_dataset = pd.read_csv('./dataset/data.csv')

dataset = encode_smiles_column_of(
    prune_dataset_lines(
        raw_dataset,
        remove_nan_lines=False,
        remove_nan_cols=True,
        remove_duplicates=True
    ),
    strategy='count_encoding'
)

X_train, y_train, X_val, y_val, X_test, y_test = get_train_data(
    dataset,
    targets_columns=['Energy_(kcal/mol)', 'Energy DG:kcal/mol)'],
    random_state=None,
    as_numpy=False
)

### Example of Optuna hyperparameters optimization with Optuna

In [3]:
rbf = RBF(n_clusters=8, sigma=3.14, normalize=True).fit(X_train, y_train)   # RBF class does data normalization no worries.
y_pred = rbf.predict(X_val)
r2_score(y_val, y_pred)

study = optuna.create_study(
    direction='maximize',
    study_name="RBF hyperparameters optimization"
)

study.optimize(
    lambda trial: objective(
        trial, X_train, y_train, X_val, y_val, normalize=True, metric=r2_score),
    n_trials=10
)

[I 2022-06-22 16:29:45,405] A new study created in memory with name: RBF hyperparameters optimization
[I 2022-06-22 16:29:46,822] Trial 0 finished with value: 0.9675792728343919 and parameters: {'n_clusters': 83, 'sigma': 8.56438780590284}. Best is trial 0 with value: 0.9675792728343919.
[I 2022-06-22 16:29:49,345] Trial 1 finished with value: 0.9918594372888045 and parameters: {'n_clusters': 233, 'sigma': 9.311741795553052}. Best is trial 1 with value: 0.9918594372888045.
[I 2022-06-22 16:29:51,858] Trial 2 finished with value: 0.9926491879453219 and parameters: {'n_clusters': 229, 'sigma': 9.103485913229711}. Best is trial 2 with value: 0.9926491879453219.
[I 2022-06-22 16:29:55,417] Trial 3 finished with value: 0.9782126125409317 and parameters: {'n_clusters': 143, 'sigma': 8.2906641403317}. Best is trial 2 with value: 0.9926491879453219.
[I 2022-06-22 16:29:57,319] Trial 4 finished with value: 0.8664031910361154 and parameters: {'n_clusters': 138, 'sigma': 3.2467175697473447}. Best

In [22]:
rbf = RBF(study.best_params["n_clusters"], study.best_params["sigma"], normalize=True).fit(X_train, y_train)
y_pred = rbf.predict(X_test)

print("Test mse =", mean_squared_error(rbf.targets_scaler.transform(y_test), rbf.targets_scaler.transform(y_pred)))
print("Test R2 =", r2_score(y_test, y_pred))
print("best parameters:", study.best_params)

Test mse = 0.0044175841721721095
Test R2 = 0.9956540474262541
best parameters: {'n_clusters': 246, 'sigma': 9.979575791093968}


/home/hugo/Projects/dm_project/.venv/lib64/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


## Performances w.r.t. training dataset size

In [ ]:
N_ESSAIS = 50
mses = [[] for _ in range(N_ESSAIS)]
r2_scores = [[] for _ in range(N_ESSAIS)]
percentages = np.linspace(0.2, 1, 50)

In [ ]:
from pathlib import Path
cluster_numbers = []
standard_deviations = []
optimization = np.arange(start=0, stop=N_ESSAIS * len(percentages), step=1)

# this took 15 min on my machine with N_ESSAIS = 5
for k in range(N_ESSAIS):
    seed = random.randint(0, 10000)
    random_state = np.random.RandomState(seed)

    for p in percentages:

        X_train, y_train, X_val, y_val, X_test, y_test = get_train_data(
            dataset,
            targets_columns=['Energy_(kcal/mol)', 'Energy DG:kcal/mol)'],
            random_state=random_state,
            as_numpy=False,
        )

        study = optuna.create_study(
            direction='maximize',
            study_name=f"RBF hyperparameters optimization for percentage p={p} Essai={k}"
        )

        study.optimize(
            lambda trial: objective(
                trial, X_train, y_train, X_val, y_val, normalize=True, metric=r2_score),
            n_trials=10,
            # n_jobs=-1
        )

        rbf = RBF(
            study.best_params["n_clusters"],
            study.best_params["sigma"],
            normalize=True
        ).fit(X_train, y_train)
        y_pred = rbf.predict(X_test)
        mses[k].append(mean_squared_error(y_test, y_pred))
        r2_scores[k].append(r2_score(y_test, y_pred))

        cluster_numbers.append(study.best_params["n_clusters"])
        standard_deviations.append(study.best_params["sigma"])


path = Path('./results/rbf')
mses = np.array(mses)
r2_scores = np.array(r2_scores)

np.save(path/'rbf_mses_var_percentage', mses)
np.save(path/'rbf_scores_var_percentage', r2_scores)


In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

path = Path('./results/rbf')

#mses = np.array(mses)
#r2_scores = np.array(r2_scores)

mses = np.load(path/'rbf_mses_var_percentage.npy')
r2_scores = np.load(path/'rbf_scores_var_percentage.npy')

np.save(path/'rbf_mses_var_percentage', mses)
np.save(path/'rbf_scores_var_percentage', r2_scores)

plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.title("Mean Squares Error")
plt.xlabel("Train Data Percentage")
plt.ylabel("Test MSE")
plt.grid(True)

final_mses = np.mean(mses, axis=0)
mses_error = np.std(mses, axis=0)

# remove the fucky percentages
rows = np.abs(final_mses) < 10000
final_mses_chosen = final_mses[rows][2:]
mses_error_chosen = mses_error[rows][2:]
percentages_chosen = percentages[rows][2:]

plt.semilogy(percentages_chosen, final_mses_chosen, label='Linear Regression')
plt.legend()
plt.fill_between(percentages_chosen, final_mses_chosen - mses_error_chosen, final_mses_chosen + mses_error_chosen, alpha=0.2, edgecolor='#1B2ACC', facecolor='#089FFF')

plt.subplot(1, 2, 2)
plt.title("R2 score")
plt.xlabel("Train Data Percentage")
plt.ylabel("Test R2 score")
plt.grid(True)
plt.yscale('linear')

final_r2_score = np.mean(r2_scores, axis=0)
r2_score_error = np.std(r2_scores, axis=0)

# remove fucky values
r2_rows = np.abs(final_r2_score) < 10000
final_r2_score_chosen = final_r2_score[r2_rows][2:]
r2_score_error_chosen = r2_score_error[r2_rows][2:]
r2_percentages_chosen = percentages[r2_rows][2:]

plt.plot(r2_percentages_chosen, final_r2_score_chosen, label='Linear Regression')
plt.legend()
plt.fill_between(r2_percentages_chosen, final_r2_score_chosen - r2_score_error_chosen, final_r2_score_chosen + r2_score_error_chosen, alpha=0.2, edgecolor='#1B2ACC', facecolor='#089FFF')

## Cross Validation Score

In [5]:
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

def cross_validation_score_of(X: pd.DataFrame, y: pd.DataFrame, n_splits=5, n_clusters=8, sigma=1.):
    kf = KFold(n_splits=n_splits)
    cross_validation_scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.to_numpy()[train_index], X.to_numpy()[test_index]
        y_train, y_test = y.to_numpy()[train_index], y.to_numpy()[test_index]

        rbf_network = RBF(n_clusters, sigma, normalize=True).fit(X_train, y_train)
        cross_validation_scores.append(r2_score(rbf_network.predict(X_test), y_test))

    return cross_validation_scores

#### Computation of Best RBF Network

In [6]:
study = optuna.create_study(
    direction='minimize',
    study_name="RBF hyperparameters optimization"
)

study.optimize(
    lambda trial: objective(
        trial, X_train, y_train, X_val, y_val, normalize=True, metric=mean_squared_error),
    n_trials=200
)

X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

final_rbf = RBF(study.best_params["n_clusters"], study.best_params["sigma"], normalize=True).fit(X_train, y_train)
y_pred = rbf.predict(X_test)

print("Test mse =", mean_squared_error(y_test, y_pred))
print("Test R2 =", r2_score(y_test, y_pred))
print("best parameters:", study.best_params)

[I 2022-06-22 16:30:28,915] A new study created in memory with name: RBF hyperparameters optimization
[I 2022-06-22 16:30:31,572] Trial 0 finished with value: 0.7453689895445778 and parameters: {'n_clusters': 247, 'sigma': 1.7258634165063231}. Best is trial 0 with value: 0.7453689895445778.
[I 2022-06-22 16:30:33,123] Trial 1 finished with value: 0.9509699672066323 and parameters: {'n_clusters': 113, 'sigma': 6.204257063667249}. Best is trial 1 with value: 0.9509699672066323.
[I 2022-06-22 16:30:34,839] Trial 2 finished with value: 0.9748475787850612 and parameters: {'n_clusters': 128, 'sigma': 8.970898925689957}. Best is trial 2 with value: 0.9748475787850612.
[I 2022-06-22 16:30:35,932] Trial 3 finished with value: 0.4854849532016037 and parameters: {'n_clusters': 38, 'sigma': 1.6029035603020458}. Best is trial 2 with value: 0.9748475787850612.
[I 2022-06-22 16:30:37,381] Trial 4 finished with value: 0.9604575143850416 and parameters: {'n_clusters': 84, 'sigma': 7.702141513517617}. B

Test mse = 1481770.7261455553
Test R2 = 0.9934639253162159
best parameters: {'n_clusters': 246, 'sigma': 9.979575791093968}


#### Computation of it's Cross Validation Score

In [ ]:
X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

cross_val_scores = cross_validation_score_of(X, y, n_clusters=246, sigma=9.979575791093968)

In [9]:
print("RBF Cross Validation Score =", np.array(cross_val_scores).mean())

RBF Cross Validation Score = 0.992251792072613


In [16]:
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

def cross_validation_mse_of(X: pd.DataFrame, y: pd.DataFrame, n_splits=5, n_clusters=8, sigma=1., denormalize=True):
    kf = KFold(n_splits=n_splits)
    cross_validation_mses = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.to_numpy()[train_index], X.to_numpy()[test_index]
        y_train, y_test = y.to_numpy()[train_index], y.to_numpy()[test_index]

        rbf_network = RBF(n_clusters, sigma, normalize=True).fit(X_train, y_train)
        
        y_pred = rbf_network.predict(X_test, denormalize)
        print(y_pred[:5, :], y_test[:5, :])
        cross_validation_mses.append(
            mean_squared_error(y_pred, rbf_network.targets_scaler.transform(y_test)))

    return cross_validation_mses

In [15]:
denormalized_cross_val_mses = cross_validation_mse_of(X, y, n_clusters=246, sigma=9.979575791093968, denormalize=True)
print("RBF Denormalized Cross Validation MSES =", np.array(denormalized_cross_val_mses).mean())

[[-60963.34585871 -60833.74487965]
 [-49527.02918189 -49495.20553857]
 [-53230.00015341 -53147.65271935]
 [-88713.81858106 -88578.21838056]
 [-34314.04111412 -34171.47736712]] [[-60722.2098  -60592.86447]
 [-50117.95156 -50086.89009]
 [-53100.40641 -53017.56812]
 [-88777.70547 -88642.7211 ]
 [-34553.75421 -34411.52583]]
[[-63677.41542205 -63572.94750295]
 [-72646.74931262 -72532.78689682]
 [-66456.18270862 -66312.75185836]
 [-71132.98664035 -71014.20911788]
 [-60283.97252628 -60176.67409425]] [[-63460.25452 -63356.92257]
 [-73173.96626 -73058.91311]
 [-66714.15885 -66571.1535 ]
 [-70820.38866 -70703.28141]
 [-60017.60798 -59910.4479 ]]
[[-51126.01633374 -51044.55505484]
 [-40919.47528464 -40800.55902644]
 [-57361.85586315 -57296.98976977]
 [-80078.48624816 -79915.91315886]
 [-61899.51817816 -61776.91785352]] [[-51451.50814 -51370.12188]
 [-41543.87978 -41425.10515]
 [-57546.44494 -57481.63395]
 [-81169.70101 -81005.92211]
 [-62439.77868 -62332.66113]]
[[-40355.62868803 -40245.90058333]

In [17]:
normalized_cross_val_mses = cross_validation_mse_of(X, y, n_clusters=246, sigma=9.979575791093968, denormalize=False)
print("RBF Normalized Cross Validation MSES =", np.array(normalized_cross_val_mses).mean())

[[-0.31883504 -0.31721915]
 [ 0.49475729  0.4912541 ]
 [ 0.22674189  0.22610923]
 [-2.1306343  -2.13161448]
 [ 1.47416501  1.47947578]] [[-60722.2098  -60592.86447]
 [-50117.95156 -50086.89009]
 [-53100.40641 -53017.56812]
 [-88777.70547 -88642.7211 ]
 [-34553.75421 -34411.52583]]
[[-0.47349976 -0.47381586]
 [-1.10502807 -1.1057558 ]
 [-0.6564791  -0.65459501]
 [-0.98372809 -0.9839143 ]
 [-0.25340236 -0.25324826]] [[-63460.25452 -63356.92257]
 [-73173.96626 -73058.91311]
 [-66714.15885 -66571.1535 ]
 [-70820.38866 -70703.28141]
 [-60017.60798 -59910.4479 ]]
[[ 0.34971992  0.3491303 ]
 [ 1.03770811  1.04078798]
 [-0.05680167 -0.05914306]
 [-1.52643679 -1.52468901]
 [-0.31153968 -0.31091283]] [[-51451.50814 -51370.12188]
 [-41543.87978 -41425.10515]
 [-57546.44494 -57481.63395]
 [-81169.70101 -81005.92211]
 [-62439.77868 -62332.66113]]
[[1.07674728 1.07927479]
 [1.90503637 1.90759913]
 [0.82579886 0.82435475]
 [0.01540869 0.0140924 ]
 [0.64106092 0.63981781]] [[-41080.27734 -40970.22519]